In [4]:
# Read data from the model and generate a time series
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from datetime import datetime, timedelta
import os                                  # Used to convert png to other format
import rpnpy.librmn.all as rmn             # Module to read RPN files
from rpnpy.rpndate import RPNDate, RPNDateRange
from datetime import datetime, timedelta
import pickle

In [19]:
'''
List of Variables:
## 2D ##
SN   : accum. of snow precip.  |  Snow amount in liquid water equivalent of falling snow. [m]
SN1  : accum. ice crystals (M-Y)  |  Accumulated ice crystals (Milbrandt-Yau) [m]
SN2  : accum. snow (M-Y)  |  Accumulated snow (Milbrandt-Yau) [m]
SN3  : accum. graupel (M-Y)  |  Accumulated graupel (Milbrandt-Yau) [m]
RN   : accum. of liquid precip.  |  Liquid precipitation amount [m]
RN1  : accum. liquid drizzle (M-Y)  |  Accumulated liquid drizzle (Milbrandt-Yau) [m]
RN2  : accum. liquid rain (M-Y)  |  Accumulated liquid rain (Milbrandt-Yau) [m]
PR   : accum. of total precip.  |  Quantity of precipitation [m]

PX: Hydrostatic Pressure
MPNC:
MPNR:
HU: Humidite specifique
MPQC:
MPQR:
QT1:
GZ: Geopotential Height
TT: Air Temperature
HR: Relative Humidity
WW: Vertical Motion
UU: Wind U
VV: Wind V
QR: Relative Vorticity
ZZ:
QTI1:
QMI1:
NTI1:
BMI1:
PN: Sea level pressure

## 3D ##
TD:
TT: Air Temperature
HU: Humidite specifique
WW: Vertical Motion
UU: Wind U
VV: Wind V
GZ: Geopotential Height
QQ: Absolute Vorticity
P0: Surface Pressure
ES: Dew Point Depression

AD: Incoming LW (Acc FI) 
FI: Incoming LW 
N4: Dowward SW (Acc FB) 
FB: Downward SW 
FC: Sensible heat flux 
NR: Net Radiation on ground 
AL: Visible surface albedo 
'''
# other ip1 value for FC: 60268832, 59868832
typ = 'pres'
typ = 'model'
var_list = ["PN","UU","VV","TT","HR","HU","SN","RN","PR","AD","FI","N4","FB","FC","NR","AL","TD","RT"]
var_list_eta = ["FV","FC","FI","FL","AL","FB","SI"]
        #  PN   UU      VV       TT      HR       HU      SN RN PR AD FI N4 FB FC      NR  AL  TD
var_ip1 = [0,75597472,75597472,76696048,76696048,76696048,0, 0, 0, 0, 0, 0, 0, 60268832,0, 60268832,76696048,0]
var_ip1_eta = [60268832,60268832,0,0,60268832,0,0]
y = 2021
m = 4
d = 22
hour=0
h = [0,6,12,18]  
h = [0]

f_hrdps = '/upslope/winger/WCPS/HRDPS_SAJESS'
f_hrdps = '/chinook/cruman/Data/HRDPS_SAJESS'

lat = 47.416899
lon = -68.325813
#lon = 291.674187

In [21]:
ii = None
jj = None
main_data = []
for hour in h:
    aux_data = []    
    aux_data_eta = []
    #aux_time = []
    dt = datetime(y, m, d, hour)

    for i, prev in enumerate(range(-30,96*30,30)):
        if i == 0:
            continue
        elif i == 1:
            aux = dt + timedelta(minutes=0)
        else:
            aux = dt + timedelta(minutes=prev)

        filename = f"{f_hrdps}/{typ}/{y}{m:02d}{d:02d}{hour:02d}_{int(i/2):03d}_zoom"        

        #print(prev, i, filename)
        
        data, ii, jj = readData(filename, var_list, var_ip1, RPNDate(aux), lat, lon, ii, jj) 
        
        filename = f"{f_hrdps}/eta/{y}{m:02d}{d:02d}{hour:02d}_{int(i/2):03d}_zoom"        
        
        data_eta, ii, jj = readData(filename, var_list_eta, var_ip1_eta, RPNDate(aux), lat, lon, ii, jj)
        
        aux_data.append(data)
        aux_data_eta.append(data_eta)
        #aux_time.append(aux)
        #aux = readData(filename, var, ip1, datev, lat, lon, i=None, j=None)
    #main_data.append((hour,aux_time,aux_data))
    df = pd.DataFrame.from_records(aux_data, columns=["Timestamp","PN","UU","VV","TT","HR","HU","SN","RN","PR","AD","FI","N4","FB","FC","NR","AL","TD","RT"])
    df_eta = pd.DataFrame.from_records(aux_data_eta, columns=["Timestamp","FV","FC","FI","FL","AL","FB","SI"])
    pickle.dump( df, open( f"data_df_{y}{m:02d}{d:02d}{hour:02d}.p", "wb" ) )
    pickle.dump( df_eta, open( f"data_eta_df_{y}{m:02d}{d:02d}{hour:02d}.p", "wb" ) )
    #sys.exit()

In [13]:
df['FI']

0     316.852112
1     316.114197
2     316.101257
3     316.157379
4     315.742218
         ...    
91    308.039978
92    309.079712
93    310.264252
94    311.280273
95    311.429749
Name: FI, Length: 96, dtype: float64

In [16]:
df_eta.dropna(axis=0)

,Timestamp,FV,FC,FI,FL,AL,FB,SI
0,2021-04-22 00:00:00+00:00,-0.785095,-4.166199,316.852112,4.078796,0.157412,0.000000,-0.798691
2,2021-04-22 01:00:00+00:00,-0.734863,-2.485413,316.101257,2.861938,0.169619,0.000000,-0.468323
4,2021-04-22 02:00:00+00:00,0.469849,-3.433838,315.742218,2.520996,0.176943,0.000000,-0.666031
6,2021-04-22 03:00:00+00:00,10.844849,-14.031433,313.033081,2.552612,0.188418,0.000000,-1.321304
8,2021-04-22 04:00:00+00:00,8.420776,-15.238525,312.028900,4.825928,0.193057,0.000000,-1.596909
10,2021-04-22 05:00:00+00:00,5.363159,-13.000854,311.051331,6.099609,0.204043,0.000000,-1.501190
12,2021-04-22 06:00:00+00:00,15.609131,-19.433289,309.165222,1.136719,0.205996,0.000000,-2.318802
14,2021-04-22 07:00:00+00:00,14.611023,-20.651855,305.534058,2.054688,0.205508,0.000000,-4.024216
16,2021-04-22 08:00:00+00:00,10.673340,-11.721069,307.000977,-0.581543,0.210635,0.000000,-1.838364
18,2021-04-22 09:00:00+00:00,16.142578,-20.880798,305.607605,2.159912,0.217471,0.000000,-2.698990


In [6]:
def readData(filename, var, ip1, datev, lat, lon, i=None, j=None):
    # Read one record
    # ---------------
    fid = rmn.fstopenall(filename,rmn.FST_RO)   # Open the file    
    data = []
    data.append(datev.toDateTime())
    for v, ip in zip(var, ip1):
        #print(v, ip, datev)
        #if v=="PN" and datev.toDateTime().minute == 30:
        #data.append(np.nan)
        #else:
        rec = rmn.fstlir(fid,nomvar=v,ip1=ip,datev=datev.datev)        # Read the full record of variable 'varname'
        if rec is None:
            data.append(np.nan)
        else:
            field = rec['d']#[i1:i2,j1:j2]                            # Assign 'field' to the data of 'varname'        
            #print(field.shape)
            
            if i is None:
                mygrid = rmn.readGrid(fid,rec)              # Get the grid information for the (LAM) Grid -- Reads the tictac's
                latlondict = rmn.gdll(mygrid)               # Create 2-D lat and lon fields from the grid information

                lat2d = latlondict['lat']#[i1:i2,j1:j2]                     # Assign 'lat' to 2-D latitude field
                lon2d = latlondict['lon']#[i1:i2,j1:j2]                    # Assign 'lon' to 2-D longitude field
                # Get it from the lat,lon from the model
                i, j = geo_idx([lat,lon], np.array([lat2d,lon2d]))
            # Get grid rotation for projection of 2-D field for mapping -  if needed
            #tics = rmn.fstlir(fid,nomvar='^^', ip1=rec['ig1'],ip2=rec['ig2'],ip3=rec['ig3']) # Read corresponding tictac's

            data.append(field[i,j])
    #sys.exit()
    # Close the RPN file
    rmn.fstcloseall(fid) 
    
    return data, i, j

In [7]:
def geo_idx(dd, dd_array, type="lat"):
  '''
    search for nearest decimal degree in an array of decimal degrees and return the index.
    np.argmin returns the indices of minium value along an axis.
    so subtract dd from all values in dd_array, take absolute value and find index of minimum.
    
    Differentiate between 2-D and 1-D lat/lon arrays.
    for 2-D arrays, should receive values in this format: dd=[lat, lon], dd_array=[lats2d,lons2d]
  '''
  if type == "lon" and len(dd_array.shape) == 1:
    dd_array = np.where(dd_array <= 180, dd_array, dd_array - 360)

  if (len(dd_array.shape) < 2):
    geo_idx = (np.abs(dd_array - dd)).argmin()
  else:
    if (dd_array[1] < 0).any():
      dd_array[1] = np.where(dd_array[1] <= 180, dd_array[1], dd_array[1] - 360)

    a = abs( dd_array[0]-dd[0] ) + abs(  np.where(dd_array[1] <= 180, dd_array[1], dd_array[1] - 360) - dd[1] )
    i,j = np.unravel_index(a.argmin(), a.shape)
    geo_idx = [i,j]

  return geo_idx

In [ ]:
'''
1) Get the date of the soundings file
2) Open the model file. Get the levels
3) Get the vars (TT, GZ, HU, HR, UU, VV, Pressure). 
4) Sort the levels
5) For each lat/lon/height in the soundings file, get the relative point in the model
6) Plot the skew-P graph
'''
# 1


# 2
a, b, c = readVerticalRecords("TT", RPNDate(dt), filename)

In [132]:
def readVerticalRecords(varname, datev, filename):
    fid = rmn.fstopenall(filename,rmn.FST_RO)
    
    record_list = []
    record_date_list = []
    #record_ip = []
    record_ip1 = []
    record_ip1_type = []
    
    for k in rmn.fstinl(fid, nomvar=varname, datev=datev.datev):
        #print(rmn.fstluk(k))
        record_list.append(rmn.fstluk(k)['d'])
        record_date_list.append(stamp2datetime(rmn.fstluk(k)['datev']))
        ip1=rmn.fstluk(k)['ip1']
        ip2=rmn.fstluk(k)['ip2']
        ip3=rmn.fstluk(k)['ip3']
        rp1, rp2, rp3 = rmn.DecodeIp(ip1, ip2, ip3)    
        #record_ip.append([(rp1.v1, rmn.kindToString(rp1.kind)), (rp2.v1, rmn.kindToString(rp2.kind)), (rp3.v1, rmn.kindToString(rp3.kind))])
        record_ip1.append(rp1.v1)
        record_ip1_type.append(rmn.kindToString(rp1.kind))    
        
    #print(record_date_list)
    
    r_data = np.array(record_list)
    r_date = np.array(record_date_list)
    r_height = np.array(record_ip1)
    
    # Sort by height
    idx = np.argsort(r_height)
    r_data = r_data[idx]
    r_date = r_date[idx]
    r_height = r_height[idx]

    return r_data[::-1], r_date[::-1], r_height[::-1]

In [45]:
# Reading 3D file
#import rpnpy.librmn.all as rmn             # Module to read RPN files
from fstd2nc.mixins.dates import stamp2datetime

f_hrdps = '/chinook/cruman/Data/HRDPS_SAJESS'
y = 2021
m = 4
d = 19
hour=0
h = [0,6,12,18] 
typ = 'model'
varname="TT"
filename = f"{f_hrdps}/{typ}/{y}{m:02d}{d:02d}{hour:02d}_001_zoom" 

fid = rmn.fstopenall(filename,rmn.FST_RO)   # Open the file

# Création de listes vides

record_list = []
record_date_list = []
record_ip = []
record_ip1 = []
record_ip1_type = []

# On fait la boucle sur tous les enregistrements de varname
# fstinf
for k in rmn.fstinl(fid, nomvar=varname, datev=datev.datev):
    #print(rmn.fstluk(k))
    record_list.append(rmn.fstluk(k)['d'])
    record_date_list.append(stamp2datetime(rmn.fstluk(k)['datev']))
    ip1=rmn.fstluk(k)['ip1']
    ip2=rmn.fstluk(k)['ip2']
    ip3=rmn.fstluk(k)['ip3']
    rp1, rp2, rp3 = rmn.DecodeIp(ip1, ip2, ip3)    
    record_ip.append([(rp1.v1, rmn.kindToString(rp1.kind)), (rp2.v1, rmn.kindToString(rp2.kind)), (rp3.v1, rmn.kindToString(rp3.kind))])
    record_ip1.append(rp1.v1)
    record_ip1_type.append(rmn.kindToString(rp1.kind))
 
# On ferme le fichier rpn
rmn.fstcloseall(fid)

In [128]:
dt = datetime(y, m, d, hour) + timedelta(minutes=30)
print(dt)
a, b, c = readVerticalRecords("TT", RPNDate(dt), filename)

2021-04-19 00:30:00


In [131]:
a.shape

(63, 601, 501)

In [108]:
# idx works on np.array and not lists.
r_data = np.array(record_list)
r_date = np.array(record_date_list)
r_height = np.array(record_ip1)

# Sort by time. Skip this if the record has only the 00 time
idx   = np.argsort(r_date)

r_data = r_data[idx]
r_date = r_date[idx]
r_height = r_height[idx]

# Split by time. 
r_data1, r_data2 = np.split(r_data, 2)
r_date1, r_date2 = np.split(r_date, 2)
r_height1, r_height2 = np.split(r_height, 2)

# Sort by height
idx   = np.argsort(r_height1)
r_data1 = r_data1[idx]
r_date1 = r_date1[idx]
r_height1 = r_height1[idx]




In [109]:
'''
1) Get the date of the soundings file
2) Open the model file. Get the levels
3) Get the vars (TT, GZ, HU, HR, UU, VV, Pressure). 
4) Sort the levels
5) For each lat/lon/height in the soundings file, get the relative point in the model
6) Plot the skew-P graph
'''

array([0.00226405, 0.00410016, 0.00742534, 0.0134471 , 0.0227437 ,
       0.0338256 , 0.0451936 , 0.0556271 , 0.0644377 , 0.0714344 ,
       0.0771276 , 0.0825822 , 0.0883959 , 0.0946183 , 0.101277  ,
       0.108407  , 0.116043  , 0.124213  , 0.13295799, 0.142318  ,
       0.152337  , 0.16306099, 0.174539  , 0.186827  , 0.19997901,
       0.21404199, 0.22904401, 0.245002  , 0.26192001, 0.279814  ,
       0.29867801, 0.31848899, 0.33922401, 0.360861  , 0.383349  ,
       0.40663299, 0.43065301, 0.45533401, 0.48060501, 0.50638801,
       0.532601  , 0.55916101, 0.58596599, 0.61293602, 0.63997698,
       0.66699898, 0.69391501, 0.72063202, 0.74708402, 0.77318698,
       0.798859  , 0.823982  , 0.84829098, 0.87148702, 0.89333802,
       0.913683  , 0.93241203, 0.94946498, 0.96484298, 0.97859901,
       0.99003798, 0.99749702, 1.5       ])

In [104]:
a

array([0.96484298, 0.383349  , 0.00410016, 0.00742534, 0.50638801,
       0.532601  , 0.823982  , 0.84829098, 0.0556271 , 0.0644377 ,
       0.21404199, 0.186827  , 0.19997901, 0.00226405, 0.279814  ,
       0.29867801, 0.55916101, 0.58596599, 0.360861  , 0.108407  ,
       0.0227437 , 0.74708402, 0.245002  , 0.26192001, 0.0946183 ,
       0.101277  , 0.61293602, 0.63997698, 0.913683  , 0.93241203,
       0.0825822 , 0.0883959 , 0.77318698, 0.798859  , 0.97859901,
       0.99003798, 0.124213  , 0.13295799, 0.72063202, 0.0134471 ,
       0.116043  , 0.22904401, 0.0338256 , 0.31848899, 0.87148702,
       0.0714344 , 0.45533401, 0.0771276 , 0.152337  , 0.142318  ,
       0.48060501, 0.89333802, 0.16306099, 0.33922401, 1.5       ,
       0.94946498, 0.174539  , 0.99749702, 0.66699898, 0.69391501,
       0.43065301, 0.40663299, 0.0451936 ])

In [69]:
list(zipped)

[]

In [44]:
record_ip[4], record_date_list[4]

([(0.1745389997959137, 'hy'), (0.5, ' H'), (0.0, ' H')],
 datetime.datetime(2021, 4, 19, 0, 30))